In [1]:
import pandas as pd

In [2]:
season_dat = pd.read_csv('data/DataFiles/RegularSeasonDetailedResults.csv')
season_dat.shape

(87504, 34)

### Advanced Metrics

Calculate:     
1) Offensive Efficiency - Possession based   
2) Defensive Efficiency - Possession based   
3) Net Rating - Possession based   
4) Assist Ratios      
5) eFG    

Maybe look for Dean Olivers Four Factor Formula 

## Regular Season Data 

In [3]:
# pos = possessions
season_dat['wPos'] = season_dat.apply(lambda row: 0.96*(row.WFGA + row.WTO + 0.44*row.WFTA - row.WOR), axis=1)
season_dat['lPos'] = season_dat.apply(lambda row: 0.96*(row.LFGA + row.LTO + 0.44*row.LFTA - row.LOR), axis=1)

In [4]:
season_dat['WOffRtg'] = season_dat.apply(lambda row: 100 * (row.WScore / row.wPos), axis=1)
season_dat['LOffRtg'] = season_dat.apply(lambda row: 100 * (row.LScore / row.lPos), axis=1)
#Defensive efficiency (DefRtg) = 100 x (Opponent points / Opponent possessions)
season_dat['WDefRtg'] = season_dat.LOffRtg
season_dat['LDefRtg'] = season_dat.WOffRtg
#Net Rating = Off.Rtg - Def.Rtg
season_dat['WNetRtg'] = season_dat.apply(lambda row:(row.WOffRtg - row.WDefRtg), axis=1)
season_dat['LNetRtg'] = season_dat.apply(lambda row:(row.LOffRtg - row.LDefRtg), axis=1)

In [5]:
#Assist Ratio : Percentage of team possessions that end in assists
season_dat['WAstR'] = season_dat.apply(lambda row: 100 * row.WAst / (row.WFGA + 0.44*row.WFTA + row.WAst + row.WTO), axis=1)
season_dat['LAstR'] = season_dat.apply(lambda row: 100 * row.LAst / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)
#Turnover Ratio: Number of turnovers of a team per 100 possessions used.
#(TO * 100) / (FGA + (FTA * 0.44) + AST + TO)
season_dat['WTOR'] = season_dat.apply(lambda row: 100 * row.WTO / (row.WFGA + 0.44*row.WFTA + row.WAst + row.WTO), axis=1)
season_dat['LTOR'] = season_dat.apply(lambda row: 100 * row.LTO / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)
                    
#eFG% : Effective Field Goal Percentage adjusting for the fact that 3pt shots are more valuable
season_dat['WeFGP'] = season_dat.apply(lambda row:(row.WFGM + 0.5 * row.WFGM3) / row.WFGA, axis=1)      
season_dat['LeFGP'] = season_dat.apply(lambda row:(row.LFGM + 0.5 * row.LFGM3) / row.LFGA, axis=1) 

In [6]:
#FTA Rate : How good a team is at drawing fouls.
season_dat['WFTAR'] = season_dat.apply(lambda row: row.WFTA / row.WFGA, axis=1)
season_dat['LFTAR'] = season_dat.apply(lambda row: row.LFTA / row.LFGA, axis=1)
                         
#OREB% : Percentage of team offensive rebounds
season_dat['WORP'] = season_dat.apply(lambda row: row.WOR / (row.WOR + row.LDR), axis=1)
season_dat['LORP'] = season_dat.apply(lambda row: row.LOR / (row.LOR + row.WDR), axis=1)
#DREB% : Percentage of team defensive rebounds
season_dat['WDRP'] = season_dat.apply(lambda row: row.WDR / (row.WDR + row.LOR), axis=1)
season_dat['LDRP'] = season_dat.apply(lambda row: row.LDR / (row.LDR + row.WOR), axis=1)                                      
#REB% : Percentage of team total rebounds
season_dat['WRP'] = season_dat.apply(lambda row: (row.WDR + row.WOR) / (row.WDR + row.WOR + row.LDR + row.LOR), axis=1)
season_dat['LRP'] = season_dat.apply(lambda row: (row.LDR + row.LOR) / (row.WDR + row.WOR + row.LDR + row.LOR), axis=1) 

In [7]:
season_dat['W4Factor'] = season_dat.apply(lambda row: .40*row.WeFGP + .25*row.WTOR + .20*row.WORP + .15*row.WFTAR, axis=1)
season_dat['L4Factor'] = season_dat.apply(lambda row: .40*row.LeFGP + .25*row.LTOR + .20*row.LORP + .15*row.LFTAR, axis=1)                                      

## Ken Pom Data

In [29]:
ken_pom = pd.read_csv('data/Final/ken_pom.csv')
ken_pom.shape

(6157, 23)

In [30]:
ken_pom.head()

,rank,team,conf,wins_losses,adj_EM,adj_offensive_efficiency,adj_offensive_efficiency_seed,adj_defensive_efficiency,adj_defensive_efficiency_seed,adj_tempo,...,sos_adj_em,sos_adj_em_seed,opposing_offenses,opposing_offenses_seed,opposing_defenses,opposing_defenses_seed,ncsos_adj_em,ncsos_adj_em_seed,year,seed
0,1,Virginia,ACC,29-3,35.54,123.6,2,88.1,5,59.2,...,10.39,30,108.4,48,98.0,17,-2.86,250,2019,1.0
1,2,Gonzaga,WCC,30-3,32.71,125.2,1,92.5,17,69.7,...,3.30,84,106.2,76,102.9,103,1.96,110,2019,1.0
2,3,Duke,ACC,29-5,31.90,120.1,6,88.2,6,72.1,...,13.13,7,110.4,5,97.3,11,5.42,38,2019,1.0
3,4,Michigan St.,B10,28-6,31.31,121.8,4,90.5,8,67.3,...,13.51,3,110.3,7,96.8,2,3.15,89,2019,2.0
4,5,Michigan,B10,28-6,29.36,115.6,18,86.2,2,64.3,...,11.55,19,109.3,28,97.8,14,-5.03,307,2019,2.0


In [31]:
df_team_spellings = pd.read_csv('data/DataFiles/TeamSpellings.csv',encoding='latin1')

In [32]:
ken_pom['team'] = ken_pom['team'].str.lower()

In [33]:
ken_pom = ken_pom.merge(df_team_spellings,left_on=['team'],right_on=['TeamNameSpelling']).drop('TeamNameSpelling',axis=1)

In [34]:
ken_pom.head()

,rank,team,conf,wins_losses,adj_EM,adj_offensive_efficiency,adj_offensive_efficiency_seed,adj_defensive_efficiency,adj_defensive_efficiency_seed,adj_tempo,...,sos_adj_em_seed,opposing_offenses,opposing_offenses_seed,opposing_defenses,opposing_defenses_seed,ncsos_adj_em,ncsos_adj_em_seed,year,seed,TeamID
0,1,virginia,ACC,29-3,35.54,123.6,2,88.1,5,59.2,...,30,108.4,48,98.0,17,-2.86,250,2019,1.0,1438
1,2,virginia,ACC,31-3,29.53,115.2,30,85.6,1,59.4,...,37,110.0,33,100.7,33,-0.08,158,2018,1.0,1438
2,12,virginia,ACC,23-11,24.29,112.2,50,88.0,2,59.2,...,10,111.5,8,99.8,22,2.29,82,2017,5.0,1438
3,4,virginia,ACC,29-8,29.64,120.6,8,91.0,7,61.3,...,2,112.1,2,99.5,6,4.98,38,2016,1.0,1438
4,6,virginia,ACC,30-4,30.06,115.6,21,85.5,2,58.0,...,32,108.5,31,100.3,37,0.46,142,2015,2.0,1438


In [35]:
kp = ken_pom[['TeamID','year','adj_offensive_efficiency','adj_defensive_efficiency','luck']]

In [36]:
kp.columns = ['TeamID','year','ADJOE','ADJDE','Luck']

In [37]:
kp.to_csv('data/Final/final_kp_dat.csv',index=False)

## Tourney Data

In [17]:
df_tourney = pd.read_csv('data/DataFiles/NCAATourneyCompactResults.csv')
df_seed = pd.read_csv('data/DataFiles/NCAATourneySeeds.csv')
print(df_tourney.shape)
print(df_seed.shape)

(2184, 8)
(2286, 3)


In [18]:
df_seed['Seed'] = df_seed['Seed'].str.replace('[a-zA-Z]', '').astype('int64')

In [19]:
df_seed.to_csv('data/Final/final_raw_seeds.csv',index=False)

In [20]:
df_seedv2 = pd.merge(df_tourney, df_seed, how='inner', 
               left_on=['Season', 'WTeamID'], 
               right_on=['Season', 'TeamID']).rename(columns={"Seed": "W_SEED"}).drop('TeamID',axis=1)
df_seedv2 = pd.merge(df_seedv2, df_seed, how='inner', 
               left_on=['Season', 'LTeamID'], 
               right_on=['Season', 'TeamID']).rename(columns={"Seed": "L_SEED"}).drop('TeamID',axis=1)

In [21]:
df_seedv2.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,W_SEED,L_SEED
0,1985,136,1116,63,1234,54,N,0,9,8
1,1985,136,1120,59,1345,58,N,0,11,6
2,1985,138,1120,66,1242,64,N,0,11,3
3,1985,136,1207,68,1250,43,N,0,1,16
4,1985,138,1207,63,1396,46,N,0,1,8


In [22]:
df_seedv2 = pd.merge(df_seedv2,kp,how='inner',
                     left_on=['Season','WTeamID'],
                     right_on = ['year','TeamID']).drop('TeamID',axis=1)
df_seedv2 = pd.merge(df_seedv2,kp,how='inner',
                     left_on=['Season','LTeamID'],
                     right_on = ['year','TeamID'],suffixes=('_W','_L')).drop('TeamID',axis=1)


In [23]:
#df_seedv2 = pd.merge(df_seedv2,ken_pom_ranks,how='inner',
                    #left_on = ['Season','WTeamID'],
                    #right_on = ['Season','TeamID']).rename(columns={'OrdinalRank':'WKP'})
#df_seedv2 = pd.merge(df_seedv2,ken_pom_ranks,how='inner',
                    #left_on = ['Season','LTeamID'],
                    #right_on = ['Season','TeamID']).rename(columns={'OrdinalRank':'LKP'})

In [24]:
df_seedv2.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,W_SEED,L_SEED,year_W,ADJOE_W,ADJDE_W,Luck_W,year_L,ADJOE_L,ADJDE_L,Luck_L
0,2002,134,1373,81,1108,77,N,0,16,16,2002,100.8,102.0,-0.070,2002,100.1,108.8,0.097
1,2002,136,1104,86,1194,78,N,0,2,15,2002,114.0,95.3,0.076,2002,99.6,102.1,0.060
2,2002,136,1112,86,1364,81,N,0,3,14,2002,119.3,98.8,0.079,2002,108.2,99.3,-0.058
3,2002,138,1112,68,1461,60,N,0,3,11,2002,119.3,98.8,0.079,2002,107.8,96.7,0.032
4,2002,136,1181,84,1457,37,N,0,1,16,2002,121.0,86.8,-0.027,2002,97.3,102.6,0.038


In [25]:
df_seedv2['Seed_Diff'] = df_seedv2['W_SEED'] - df_seedv2['L_SEED']
df_seedv2['ADJ_OE_Diff'] = df_seedv2['ADJOE_W'] - df_seedv2['ADJOE_L']
df_seedv2['ADJ_DE_Diff'] = df_seedv2['ADJDE_W'] - df_seedv2['ADJDE_L']
df_seedv2['Luck_Diff'] = df_seedv2['Luck_W'] - df_seedv2['Luck_L']
#df_seedv2['KP_Diff'] = df_seedv2['WKP'] - df_seedv2['LKP']

In [26]:
df_seedv3 = df_seedv2[df_seedv2['Season']>=2003]

In [27]:
df_wins = pd.DataFrame()
df_wins['Seed_Diff'] = df_seedv3['Seed_Diff']
df_wins['ADJ_OE_DIFF'] = df_seedv3['ADJ_OE_Diff']
df_wins['ADJ_DE_DIFF'] = df_seedv3['ADJ_DE_Diff']
df_wins['Luck_DIFF'] = df_seedv3['Luck_Diff']
df_wins['result'] = 1

df_losses = pd.DataFrame()
df_losses['Seed_Diff'] = -df_seedv2['Seed_Diff']
df_losses['ADJ_OE_DIFF'] = -df_seedv3['ADJ_OE_Diff']
df_losses['ADJ_DE_DIFF'] = -df_seedv3['ADJ_DE_Diff']
df_losses['Luck_DIFF'] = -df_seedv3['Luck_Diff']
df_losses['result'] = 0

df_seed_preds = pd.concat((df_wins, df_losses))

In [28]:
df_seed_preds.to_csv('data/Final/data_kp_seed.csv',index=False)